In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [2]:
train_df = pd.read_csv('./fe_v2_train_076.csv')
test_df = pd.read_csv('./fe_v2_test_076.csv')

In [3]:
exclude_cols = ['index', 'startdate']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]

In [4]:
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 129


In [5]:
group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']

In [6]:
best_cols = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5']

In [7]:
print("Len:", len(best_cols))

Len: 116


In [8]:
def assign_train_test_df(df, my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()
    return df

def create_df_by_group_and_agg(dataframe, grouped_cols, action, my_col, return_name):
    if action == "first":
        ans_df = dataframe.groupby(grouped_cols)[my_col].first().reset_index()
    elif action == "min":
        ans_df = dataframe.groupby(grouped_cols)[my_col].min().reset_index()
    elif action == "max":
        ans_df = dataframe.groupby(grouped_cols)[my_col].max().reset_index()
    elif action == "mean":
        ans_df = dataframe.groupby(grouped_cols)[my_col].mean().reset_index()
    print("Nan")
    ans_df = ans_df.rename(columns={my_col: return_name})
    for c in ans_df.columns:
        if ans_df[c].isnull().sum() > 0:
            print(c, ans_df[c].isnull().sum())
    return ans_df

def create_feature(df, alias_name, col_name, suffix_num, season_group_cols):
    first_df = create_df_by_group_and_agg(df, season_group_cols, "first", col_name, f"first_{alias_name}_{suffix_num}")
    min_df = create_df_by_group_and_agg(df, season_group_cols, "min", col_name, f"min_{alias_name}_{suffix_num}")
    max_df = create_df_by_group_and_agg(df, season_group_cols, "max", col_name, f"max_{alias_name}_{suffix_num}")
    
    df = df.merge(first_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_first'] = df[col_name] - df[f"first_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_first')
    
    df = df.merge(min_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_min'] = df[col_name] - df[f"min_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_min')
    
    df = df.merge(max_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_max'] = df[col_name] - df[f"max_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_max')
    
    df[f'range_{alias_name}_{suffix_num}'] = df[f"max_{alias_name}_{suffix_num}"] - df[f"min_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'range_{alias_name}_{suffix_num}')
    
    df[f'scale_{alias_name}_{suffix_num}'] = (df[col_name] - df[f"min_{alias_name}_{suffix_num}"]) / df[f'range_{alias_name}_{suffix_num}']
    df = assign_train_test_df(df, f'scale_{alias_name}_{suffix_num}')
    
    return [df, f'diff_{alias_name}_{suffix_num}_first',
   f'diff_{alias_name}_{suffix_num}_min',
    f'diff_{alias_name}_{suffix_num}_max',
    f'range_{alias_name}_{suffix_num}', f'scale_{alias_name}_{suffix_num}']

In [21]:
slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
poten_col = 'contest-pevpr-sfc-gauss-14d__pevpr'
rhum_col = 'contest-rhum-sig995-14d__rhum'
elevation_col = 'elevation__elevation'
wind_col_10 = 'contest-wind-h10-14d__wind-hgt-10'
sst_1_col = 'sst-2010-1'

In [10]:
df = pd.concat([train_df[main_attrs], test_df[main_attrs]], axis=0)
df.shape

(407088, 129)

In [11]:
pres_res = create_feature(df, 'pres', pres_col, 1, season_group_cols)
df = pres_res[0]
pres_feature = pres_res[1:]

Nan
Nan
Nan


In [18]:
slp_res = create_feature(df, 'slp', slp_col, 1, season_group_cols)
df = slp_res[0]
slp_feature = slp_res[1:]

Nan
Nan
Nan


In [12]:
precip_res = create_feature(df, 'precip', precip_col, 1, season_group_cols)
df = precip_res[0]
precip_feature = precip_res[1:]

Nan
Nan
Nan


In [22]:
sst_1_month_res = create_feature(df, 'sst_1_month', sst_1_col, 1, group_cols)
df = sst_1_month_res[0]
sst_1_month_feature = sst_1_month_res[1:]

Nan
Nan
Nan


In [13]:
wind_10_res = create_feature(df, 'wind_10', wind_col_10, 1, season_group_cols)
df = wind_10_res[0]
wind_10_feature = wind_10_res[1:]

Nan
Nan
Nan


In [14]:
wind_10_res_month = create_feature(df, 'wind_10_month', wind_col_10, 1, group_cols)
df = wind_10_res_month[0]
wind_10_month_feature = wind_10_res_month[1:]

Nan
Nan
Nan


In [15]:
modified_best_cols = [c for c in best_cols]

In [27]:
features = modified_best_cols + pres_feature + sst_1_month_feature

In [28]:
features

['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__c

In [29]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
res_df = pd.read_csv('./sample_solution.csv')
res_df['contest-tmp2m-14d__tmp2m'] = s
res_df.to_csv('submission_fe_v6.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:01,  3.71it/s]


(375734, 126) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.072699
0:	learn: 9.2611105	test: 9.2407165	best: 9.2407165 (0)	total: 7.29ms	remaining: 36.4s
200:	learn: 1.1772832	test: 1.1835740	best: 1.1835740 (200)	total: 1.13s	remaining: 26.9s
400:	learn: 0.9140162	test: 0.9234808	best: 0.9234808 (400)	total: 2.13s	remaining: 24.4s
600:	learn: 0.7870317	test: 0.7990677	best: 0.7990677 (600)	total: 3.16s	remaining: 23.1s
800:	learn: 0.7026516	test: 0.7162924	best: 0.7162924 (800)	total: 4.23s	remaining: 22.2s
1000:	learn: 0.6442753	test: 0.6593547	best: 0.6593547 (1000)	total: 5.25s	remaining: 21s
1200:	learn: 0.5989021	test: 0.6151854	best: 0.6151854 (1200)	total: 6.24s	remaining: 19.7s
1400:	learn: 0.5640462	test: 0.5815525	best: 0.5815525 (1400)	total: 7.25s	remaining: 18.6s
1600:	learn: 0.5325581	test: 0.5510269	best: 0.5510269 (1600)	total: 8.22s	remaining: 17.5s
1800:	learn: 0.5074423	test: 0.5269623	best: 0.5269623 (1800)	total: 9.3s	remaining: 16.5s
2000:	learn: 0.4864530	test: 0.5069317	best: 0.5069317 (2000)	tota

 20%|█████████████████▌                                                                      | 1/5 [00:37<02:30, 37.54s/it]

...

Learning rate set to 0.072699
0:	learn: 9.2604847	test: 9.2566518	best: 9.2566518 (0)	total: 5.96ms	remaining: 29.8s
200:	learn: 1.1609247	test: 1.1761170	best: 1.1761170 (200)	total: 1.08s	remaining: 25.9s
400:	learn: 0.9128861	test: 0.9284839	best: 0.9284839 (400)	total: 2.17s	remaining: 24.9s
600:	learn: 0.7833182	test: 0.7991784	best: 0.7991784 (600)	total: 3.25s	remaining: 23.8s
800:	learn: 0.7010969	test: 0.7173244	best: 0.7173244 (800)	total: 4.23s	remaining: 22.2s
1000:	learn: 0.6425653	test: 0.6593631	best: 0.6593631 (1000)	total: 5.26s	remaining: 21s
1200:	learn: 0.5982812	test: 0.6158765	best: 0.6158765 (1200)	total: 6.21s	remaining: 19.6s
1400:	learn: 0.5623325	test: 0.5808163	best: 0.5808163 (1400)	total: 7.25s	remaining: 18.6s
1600:	learn: 0.5324432	test: 0.5520789	best: 0.5520789 (1600)	total: 8.25s	remaining: 17.5s
1800:	learn: 0.5081601	test: 0.5286396	best: 0.5286396 (1800)	total: 9.28s	remaining: 16.5s
2000:	learn: 0.4859665	test: 0.5072204	best: 0.5072204 (2000)	tot

 80%|██████████████████████████████████████████████████████████████████████▍                 | 4/5 [02:31<00:37, 37.96s/it]

Learning rate set to 0.072699
0:	learn: 9.2625918	test: 9.2370575	best: 9.2370575 (0)	total: 6.68ms	remaining: 33.4s
200:	learn: 1.1720694	test: 1.1769349	best: 1.1769349 (200)	total: 1.03s	remaining: 24.5s
400:	learn: 0.9157651	test: 0.9234170	best: 0.9234170 (400)	total: 2s	remaining: 23s
600:	learn: 0.7887940	test: 0.7978129	best: 0.7978129 (600)	total: 2.98s	remaining: 21.8s
800:	learn: 0.7052048	test: 0.7156928	best: 0.7156928 (800)	total: 3.97s	remaining: 20.8s
1000:	learn: 0.6446204	test: 0.6564607	best: 0.6564607 (1000)	total: 4.93s	remaining: 19.7s
1200:	learn: 0.5979105	test: 0.6110703	best: 0.6110703 (1200)	total: 5.96s	remaining: 18.8s
1400:	learn: 0.5620670	test: 0.5765016	best: 0.5765016 (1400)	total: 7.02s	remaining: 18s
1600:	learn: 0.5320831	test: 0.5479938	best: 0.5479938 (1600)	total: 8.05s	remaining: 17.1s
1800:	learn: 0.5076898	test: 0.5245161	best: 0.5245161 (1800)	total: 9.08s	remaining: 16.1s
2000:	learn: 0.4849542	test: 0.5027704	best: 0.5027704 (2000)	total: 1

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 18.06it/s]


cos_sim with best submission: 0.9999544848739318
